In [291]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(6012)

In [292]:
def open_path(path):
    datalist=[]
    with open(path) as f:
        for line in f:
            line_list = line.split()
            datalist.append(line_list)
        new_list = []
        new_datalist = []
        for lst in datalist:
            for val in lst:
                val = float(val)
                new_list.append(val)
            new_datalist.append(new_list)
            new_list = []
        featurelist = []
        if path == "y_train.txt":
            new_datalist = sum(new_datalist, [])
            pass
        else:
            for row in new_datalist:
                array = np.asarray(row).astype('float32')
                featurelist.append(array)    
    return featurelist, new_datalist

In [334]:
x_acc_train = open_path("body_acc_x_train.txt")[0]
y_acc_train = open_path("body_acc_y_train.txt")[0]
z_acc_train = open_path("body_acc_z_train.txt")[0]
x_gyro_train = open_path("body_gyro_x_train.txt")[0]
y_gyro_train = open_path("body_gyro_y_train.txt")[0]
z_gyro_train = open_path("body_gyro_z_train.txt")[0]
total_acc_x_train = open_path("total_acc_x_train.txt")[0]
total_acc_y_train = open_path("total_acc_y_train.txt")[0]
total_acc_z_train = open_path("total_acc_z_train.txt")[0]



subject_train = open_path("subject_train.txt")[0]
y_train = open_path("y_train.txt")[1]



In [333]:
type(x_acc_train)


list

In [318]:
x_acc_test = open_path("body_acc_x_test.txt")[0]
y_acc_test = open_path("body_acc_y_test.txt")[0]
z_acc_test = open_path("body_acc_z_test.txt")[0]
x_gyro_test = open_path("body_gyro_x_test.txt")[0]
y_gyro_test = open_path("body_gyro_y_test.txt")[0]
z_gyro_test = open_path("body_gyro_z_test.txt")[0]
y_test = open_path("y_test.txt")[1]



In [335]:
stacked_arrays = np.stack((x_acc_train, y_acc_train, z_acc_train, 
                           x_gyro_train, y_gyro_train, z_gyro_train,
                           total_acc_x_train, total_acc_y_train, total_acc_z_train))

In [336]:
import torch

In [337]:
import tensorflow as tf

In [370]:
def LSTMready(X):
    
    X = (X - np.min(X)) / (np.max(X) - np.min(X))
    X = X.reshape(len(X), 128, 1)
    return X

In [371]:
x_acc_LSTMready = LSTM_ready(x_acc_train)
y_acc_LSTMready = LSTM_ready(y_acc_train)
z_acc_LSTMready = LSTM_ready(z_acc_train)
x_gyro_LSTMready = LSTM_ready(x_gyro_train)
y_gyro_LSTMready = LSTM_ready(y_gyro_train)
z_gyro_LSTMready = LSTM_ready(z_gyro_train)
total_acc_x_LSTMready = LSTM_ready(total_acc_x_train)
total_acc_y_LSTMready = LSTM_ready(total_acc_y_train)
total_acc_z_LSTMready = LSTM_ready(total_acc_z_train)

In [372]:
train_stack = np.stack((x_acc_LSTMready, y_acc_LSTMready, z_acc_LSTMready, 
                        x_gyro_LSTMready, y_gyro_LSTMready, z_gyro_LSTMready,
                        total_acc_x_LSTMready, total_acc_y_LSTMready, total_acc_z_LSTMready))

In [373]:
np.shape(train_stack)

(9, 7352, 128, 1)

In [374]:
train_stack = train_stack.reshape(7352, 128, 9)

In [375]:
np.shape(train_stack)

(7352, 128, 9)

In [376]:
y = np.array(y_train)

In [377]:
y_tf = tf.keras.utils.to_categorical(y-1, num_classes = 6)

In [379]:
X = train_stack


y = y_tf

model = Sequential()
model.add(tf.keras.layers.Dropout(rate=0.25))
model.add(LSTM(50, return_sequences=False, input_shape=(X.shape[1], X.shape[2])))
model.add(tf.keras.layers.Dropout(rate=0.25))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='mse')

model.fit(X, y, batch_size=500, epochs=10)

Epoch 1/10
15/15 [==============================] - 11s 445ms/step - loss: 1.8116 - accuracy: 0.1798
Epoch 2/10
15/15 [==============================] - 7s 436ms/step - loss: 1.8052 - accuracy: 0.1857
Epoch 3/10
15/15 [==============================] - 7s 456ms/step - loss: 1.8005 - accuracy: 0.1862
Epoch 4/10
15/15 [==============================] - 7s 498ms/step - loss: 1.7967 - accuracy: 0.1764
Epoch 5/10
15/15 [==============================] - 7s 485ms/step - loss: 1.7956 - accuracy: 0.1741
Epoch 6/10
15/15 [==============================] - 7s 485ms/step - loss: 1.7907 - accuracy: 0.1840
Epoch 7/10
15/15 [==============================] - 7s 460ms/step - loss: 1.7914 - accuracy: 0.1817
Epoch 8/10
15/15 [==============================] - 8s 500ms/step - loss: 1.7913 - accuracy: 0.1827
Epoch 9/10
15/15 [==============================] - 8s 525ms/step - loss: 1.7900 - accuracy: 0.1741
Epoch 10/10
15/15 [==============================] - 7s 477ms/step - loss: 1.7893 - accuracy: 0.183

In [314]:
predictions = model.predict(X)

In [315]:
predictions

array([[0.47971213, 0.45224988, 0.44907835, 0.50356084, 0.5183544 ,
        0.5298096 ],
       [0.4797316 , 0.45230243, 0.4491513 , 0.5035473 , 0.5183367 ,
        0.5297919 ],
       [0.47968116, 0.45224112, 0.44900453, 0.5035372 , 0.5183594 ,
        0.5298216 ],
       ...,
       [0.47408712, 0.4434974 , 0.42995024, 0.50368696, 0.5211948 ,
        0.53222454],
       [0.4755715 , 0.44486177, 0.4356188 , 0.50352585, 0.52111673,
        0.5331241 ],
       [0.47968003, 0.45272675, 0.45041487, 0.5028998 , 0.5183965 ,
        0.53075194]], dtype=float32)

In [286]:
pred_list = []
for arr in predictions:
    pred_probs = arr.tolist()
    pred = pred_probs.index(max(pred_probs))
    pred_list.append(int(pred)+1)
    
